In [1]:
import pandas as pd
import numpy as np
d=pd.read_excel('D:\GoogleDrive\custom Codes\Mar18\Collateral\Loan_D1.xlsx')

In [2]:
#d.head()

In [3]:
# get the column names
#d.columns

In [4]:
# isin 
d1=d[(d['SBS_SECURITY_CODE'].isin([73, 75, 76]))]
#d1.head()

In [43]:
l1=pd.Series(range(606, 630))
l2=pd.Series(range(634, 714))
l3=pd.Series(range(720,751))

l=l1.append(l2).append(l3)

# is not in ~
d1=d1[(~d1['PRODUCT_ID'].isin(l))]
d1.shape

(168, 26)

In [5]:
# crosstab
#pd.crosstab(index=d1['PRODUCT_ID'], columns='count')

In [6]:
p=pd.pivot_table(d1, index=['CUSTOMER_ID'], values='CLOSE_BALA_LCY', aggfunc=[len, sum])

# modifying the column names
p.columns = p.columns.to_series().str.join('_')

# convert to regular data frame
p = p.reset_index().rename_axis(None, axis=1)
p1=p.copy()
p1['sum_CLOSE_BALA_LCY']=p1['sum_CLOSE_BALA_LCY']/10000000

# sort a column
p1=p1.sort_values(by='sum_CLOSE_BALA_LCY', ascending=False)

#make the customer id column string
p1['CUSTOMER_ID']=p1['CUSTOMER_ID'].astype(str)

# check whether the column is string or not
from pandas.api.types import is_string_dtype
is_string_dtype(p1['CUSTOMER_ID'])

#p1

True

In [7]:
# add leading 0 to make 10 character string, 12 is put here as theere is a decimal and a 0 after
p1['CUSTOMER_ID']=p1['CUSTOMER_ID'].apply(lambda x: x.zfill(12))

# substr and remove the decimal and zero
p1['CUSTOMER_ID']=p1['CUSTOMER_ID'].str[:10]

# rename the columns
p1=p1.rename(columns={'len_CLOSE_BALA_LCY':'NO_OF_AC'})

# make the account no column integer
p1['NO_OF_AC']=p1['NO_OF_AC'].astype(int)

# make the index ordered
p1.index=np.arange(0, len(p1))
#p1

In [47]:
# turn d1 customer id string, fill upto 12 and then keep only the first 10 characters
d1['CUSTOMER_ID']=d1['CUSTOMER_ID'].astype(str).apply(lambda x: x.zfill(12)).str[:10]

In [8]:
# merge the two data frames
dd=pd.merge(p1, d1, on='CUSTOMER_ID', how='left')

# keep the desired columns and remove the duplicates
dd=dd.iloc[:,[0, 7, 1, 2]].drop_duplicates()

# rename columns
dd=dd.rename(columns={'len_CLOSE_BALA_LCY':'NO OF AC', 'sum_CLOSE_BALA_LCY':'TOTAL BALANCE'})

#dd